<a href="https://colab.research.google.com/github/radza99/Lab/blob/main/SentimentAnalysisBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ใช้ DecisionTreeClassifier และ เปลี่ยน data set**

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.tree import DecisionTreeClassifier  #DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# โหลดข้อมูล
df = pd.read_csv("my_sentiment_data.csv")

In [15]:
df.head(10)

,text,sentiment
0,ฉันมีความสุขมากวันนี้,positive
1,ไม่โอเคเลย ผิดหวังมาก,negative
2,เฉย ๆ ไม่ดีไม่แย่,neutral
3,ชอบมากเลย,positive
4,เบื่อจัง ไม่อยากทำอะไรเลย,negative
5,วันนี้อากาศดีมาก,positive
6,รู้สึกแย่กับเรื่องนี้,negative
7,ธรรมดา ไม่มีอะไรพิเศษ,neutral
8,รักประเทศไทยมาก,positive
9,หงุดหงิดสุด ๆ,negative


In [4]:
# โหลด tokenizer และ model ของ WangchanBERTa
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModel.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

In [14]:
# ฟังก์ชันดึง embedding
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [6]:
X = df["text"].apply(get_embedding).tolist()
X = torch.tensor(X)
y = df["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y, test_size=0.2, random_state=42)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `CamembertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipython-input-1110699426.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X = torch.tensor(X)


In [7]:
# ✅ ใช้ Decision Tree แทน Logistic Regression
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [8]:
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [11]:
new_text = ["วันนี้อากาศดีมาก"]
inputs = tokenizer(new_text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    new_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().reshape(1, -1)
predicted_sentiment = classifier.predict(new_embedding)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `CamembertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [12]:
print("\nNew Text Prediction:")
print("Text:", new_text[0])
print("Predicted Sentiment:", predicted_sentiment[0])


New Text Prediction:
Text: วันนี้อากาศดีมาก
Predicted Sentiment: positive
